# install

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Dec 15 10:53:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    56W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
!pip3 install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 23.4 MB/s 
     |████████████████████████████████| 280 kB 90.8 MB/s 
     |████████████████████████████████| 1.6 MB 70.9 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16321 sha256=53742613efebc191291cdf682cf5f833d31925bdfe07fde3290f305636d70cf8
  Stored in directory: /root/.cache/pip/wheels/08/66/b4/921e351e63d88696932279d6163e125727c9da70ed8ca38419
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


# Data Load

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Hackathon/src/preprocessing')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Hackathon/src/preprocessing']


In [6]:
!ls /content/drive/MyDrive/Hackathon/src/preprocessing

config.py      outputs		    __pycache__
data_merge.py  preprocessing.ipynb  tokenizer_BPE.py


In [7]:
import config
import json
import pandas as pd

In [8]:
# json
def load_jsonl(path):
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    data = pd.DataFrame(data)
    print("Loaded {} records from {}".format(len(data), path))
    return data


In [9]:
df_train = load_jsonl(config.train_data)
df_val = load_jsonl(config.val_data)
df_test = load_jsonl(config.test_data)

Loaded 342390 records from drive/MyDrive/Hackathon/data/train.json
Loaded 10942 records from drive/MyDrive/Hackathon/data/val.json
Loaded 4494 records from drive/MyDrive/Hackathon/data/test.json


# Preprocessing

##### 민감정보 마스킹

##### 공백 여러개 제거

In [10]:
import re
import swifter

In [11]:
df_train['utterance'] = df_train.utterance.swifter.apply(lambda x: re.sub(' +', ' ', str(x)))

Pandas Apply:   0%|          | 0/342390 [00:00<?, ?it/s]

In [12]:
df_val['utterence'] = df_val.utterance.swifter.apply(lambda x: re.sub(' +', ' ', str(x)))

Pandas Apply:   0%|          | 0/10942 [00:00<?, ?it/s]

In [13]:
df_test['utterence'] = df_test.utterance.swifter.apply(lambda x: re.sub(' +', ' ', str(x)))

Pandas Apply:   0%|          | 0/4494 [00:00<?, ?it/s]

##### 한명이 여러번 말한 경우 합치기

check

In [16]:
data = []
check = ['']
for i in df_train.iterrows():
  if check[i[0]-1] == i[1]['speaker']:
    print(check[i[0]-1])
    print(i[1]['speaker'])
    print(f"이전 문장 : {data[i[0]-1][2]}")
    print(f"현재 문장 : {i[1]['utterance']}")
    print(data)
    data[i[0]][2] = data[i[0]-1][2] + " " + i[1]['utterance']
  else:
    temp = [i[1]['conv_no'], i[1]['speaker'], i[1]['utterance'], i[1]['summary']]
    data.append(temp)
  check.append(i[1]['speaker'])

speaker1
speaker1
이전 문장 : 반갑습니다. 저는 30대 남자입니다.
현재 문장 : 저는 통역가입니다. 하시는일이 있으신가요?
[[1, 'speaker1', '처음뵙겠습니다. 저는 20대 여성입니다', '나는 20대 여성이다.'], [1, 'speaker2', '반갑습니다. 저는 30대 남자입니다.', '나는 30대 남자이다']]


IndexError: ignored

In [ ]:
df_train.iloc[2515:2530]

,speaker,personaID,utterance,summary,terminate,conv_no
2515,speaker1,12520,그렇군요 저는 tv를 잘 안봐요 학생이라서 공부하느라 바쁘거든요,나는 tv를 잘 보지 않는다.\n나는 공부하느라 바쁘다.\n나는 학생이다.,false,163
2516,speaker2,26081,아무래도 그렇겠네요. 졸릴 땐 어떻게 하세요? 제가 학생 때는 커피 많이 마시지 말...,나는 커피 없이 살 수가 없다. 나는 아이스 아메리카노를 매일 2잔 이상 마시고 있다.,false,163
2517,speaker1,12520,졸릴때는 찬물로 세안하거나 양치를 해요!,나는 졸릴 때 찬물 세안을 하거나 양치를 한다.,false,163
2518,speaker2,26081,그러시군요. 좋아하는 색은 있으세요?,,false,163
2519,speaker1,12520,저는 노란색을 좋아해요! 무슨색 좋아하세요?,나는 노란색을 좋아한다.,false,163
2520,speaker2,26081,"엇 저는 가리는 색은 없는데, 노란색을 싫어해요... 색 취향은 안 맞네요.",나는 노란색을 싫어한다.,false,163
2521,speaker1,12520,그렇군요 .. 저는 이제 학교 끝나고 집에 왔어요 친구가 많지 않아서 매일 두렵네요,나는 매일 두렵다,false,163
2522,speaker2,26081,10대 때는 학교에서 친구들과 매일 붙어있다 보니까 친구 관계가 정말 크게 느껴지더...,나는 다른 사람을 잘 배려하는 편이다.,false,163
2523,speaker1,12520,고맙습니다. 도서관 사서이시다 보니까 말하는 것에서 책을 정말 많이 읽은 게 느껴져...,나는 위인전기 읽는 것을 싫어한다,false,163
2524,speaker2,26081,그렇게 봐주시다니 감사해요. @이름@님도 고민하는 만큼 단단한 어른으로 자라실 거예요.,,false,163
